In [28]:
# coding:utf-8
import requests
import hashlib
import urllib2
import urllib
import urlparse
from bs4 import BeautifulSoup
import datetime
from urllib import quote
import time

def crawl(url):
    user_agent = 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.86 Safari/537.36'
    headers = {'User-Agent': user_agent}
    request = urllib2.Request(url, headers=headers)
    response = urllib2.urlopen(request) 
    html = response.read()
    soup = BeautifulSoup(html, from_encoding='utf-8')
    return soup

def parse(soup):
    results = soup.find('div', id='content_left').find_all('div', class_='result')
    ans = []
    for result in results:
        # title: 新闻标题
        # title_url: 新闻链接
        # source_and_time: 作者/时间
        # source: 作者
        # time: 时间
        # summary: 摘要
        # simi_words: 相同新闻
        # simi_words_url: 相同新闻查询的url
        # simi_words_search: cont:....
        title = result.find('h3', class_='c-title').find('a').get_text()
        title_url = result.find('h3', class_='c-title').find('a')['href']
        try:
            abstract = result.find('div', class_='c-summary c-row ').get_text()
        except:
            abstract = result.find('div', class_='c-span18 c-span-last').get_text()
        try:
            source_and_time = result.find('p', class_='c-author').get_text()
            source, time = source_and_time.split(u'\xa0'u'\xa0')
        except:
            source_and_time = ""
            source = ""
            time = ""
        temp_abstract = abstract[len(source_and_time):]
        summary = ""
        for word in temp_abstract:
            summary = summary + word
            if summary[-3:] == '...':
                break
        temp_list = [title, title_url, source, time, summary]
        temp = " "
        try:
            simi_words = result.find('a', class_='c-more_link').get_text()
            simi_words_url = 'http://news.baidu.com' + result.find('a', class_='c-more_link')['href']
            i = 0
            while i != len(simi_words_url) - 1:
                if simi_words_url[i] == '+':
                    while simi_words_url[i] != '&':
                        temp = temp + simi_words_url[i]
                        i = i + 1
                else:
                    i = i + 1
        except:
            simi_words = ""
            simi_words_url = ""
            temp = ""
        temp_list.append(simi_words)
        temp_list.append(simi_words_url)
        temp_list.append(temp)
        ans.append(temp_list)

#     if is_homepage == 1:
#         page_list = []
#         try:    
#             pages = soup.find('p', id='page').find_all('a')
#             for page in pages:
#                 page_list.append('http://news.baidu.com' + page['href'])
#         except:
#             pass

#         return page_list, ans
#     else:
    return ans

# def search(query_word):
#     word = urllib.quote(query_word)
#     url  = 'http://news.baidu.com/ns?cl=2&tn=news&word=' + word
#     soup = crawl(url)
#     page_list, ans = parse(soup, 1)
#     return page_list, ans

def search(query_word):
    word = urllib.quote(query_word)
    url  = 'http://news.baidu.com/ns?cl=2&tn=news&word=' + word
    return url

def get_more(page_list):
    ans = []
    for page in page_list:
        soup = crawl(page)
        temp_ans = parse(soup, 0)
        ans = ans + temp_ans

    return ans

def get_same(url):
    soup = crawl(url)
    ans = parse(soup, 0)
    return ans

def date_filter(begin_date, end_date, query_word):
    date0 = datetime.datetime.strptime(begin_date, "%Y-%m-%d")
    date1 = datetime.datetime.strptime(end_date + " 23:59:59", "%Y-%m-%d %H:%M:%S")
    y0, m0, d0 = str(date0.year), str(date0.month), str(date0.day)
    y1, m1, d1 = str(date1.year), str(date1.month), str(date1.day)
    query_word = quote(query_word)
    bt = str(int(time.mktime(date0.timetuple())))
    et = str(int(time.mktime(date1.timetuple())))
    url = 'http://news.baidu.com/ns?from=news&cl=2&bt='+ bt + '&y0='+ y0 +'&m0=' + m0 + '&d0=' + d0 + '&y1=' + y1 + '&m1=' + m1 + '&d1=' + d1 + '&et=' + et + '&q1=' + query_word + '&submit=%B0%D9%B6%C8%D2%BB%CF%C2&q3=&q4=&mt=0&lm=&s=2&begin_date=' + begin_date + '&end_date=' + end_date + '&tn=newsdy&ct1=1&ct=1'
    return url

def page_filter(url, page=0):# append url with pn and rn params
    # rn are set to be 20
    if page is None:
        page = 0
    rn = 20
    pn = page * rn
    rn = str(rn)
    pn = str(pn)
    url = url + '&pn=' + pn
    url = url + '&rn=' + rn
    return url

def search_with_page(query_word, page=0):
    url = search(query_word)
    url = page_filter(url, page)
    return url

def date_filter_with_page(begin_date, end_date, query_word, page=0):
    url = date_filter(begin_date, end_date, query_word)
    url = page_filter(url, page)
    return url

In [22]:
plist, ans = search('军民') #测试搜索词语

In [23]:
ans[0]

[u'\u6d77\u519b\u4e0e\u6d77\u6d0b\u5c40\u7b7e\u7f72\u5408\u4f5c\u534f\u8bae \u63a8\u52a8\u519b\u6c11\u878d\u5408',
 'http://finance.sina.com.cn/roll/2017-04-21/doc-ifyepsra4982970.shtml',
 u'\u65b0\u6d6a\u8d22\u7ecf',
 u'26\u5206\u949f\u524d',
 u'\u4e2d\u56fd\u8bc1\u5238\u7f51\u8baf\u6d77\u519b\u4e0e\u56fd\u5bb6\u6d77\u6d0b\u5c4021\u65e5\u5728\u4eac\u7b7e\u7f72\u519b\u6c11\u878d\u5408\u521b\u65b0\u53d1\u5c55\u6218\u7565\u5408\u4f5c\u6846\u67b6\u534f\u8bae,\u53cc\u65b9\u5546\u5b9a\u5728\u6d77\u6d0b\u6218\u7565\u89c4\u5212\u3001\u653f\u7b56\u6cd5\u89c4\u3001\u7efc\u5408\u4fdd\u969c\u3001\u7efc\u5408\u7ba1\u7406\u3001\u6d77\u4e0a\u7ef4\u6743\u6267\u6cd5\u3001\u4eba\u624d\u57f9\u517b\u4e0e\u4ea4\u6d41...',
 u'10\u6761\u76f8\u540c\u65b0\u95fb',
 'http://news.baidu.com/ns?word=%E5%86%9B%E6%B0%91+cont:2174988303&same=10&cl=1&tn=news&rn=30&fm=sd',
 ' +cont:2174988303']

In [24]:
# 测试相同新闻
cnt = 0
for elem in ans[3]:
    print str(cnt) + ': ' + elem
    cnt = cnt + 1

0: 全市军民融合发展推进会在上杭召开
1: http://www.ctw.cn/article/article_127512.html
2: 长汀网
3: 1小时前
4: 昨日下午,全市军民融合发展推进会在上杭召开,回顾总结去年全市军民融合发展工作,传达贯彻市军民融合发展领导小组第一次会议精神,研究部署安排下一步工作。副市长谢海波...
5: 
6: 
7: 


In [25]:
print ans[0][6]
temp = " "
i = 0
while i != len(ans[0][6]) - 1:
    if ans[0][6][i] == '+':
        while ans[0][6][i] != '&':
            temp = temp + ans[0][6][i]
            i = i + 1
    else:
        i = i + 1
print temp

http://news.baidu.com/ns?word=%E5%86%9B%E6%B0%91+cont:2174988303&same=10&cl=1&tn=news&rn=30&fm=sd
 +cont:2174988303


In [26]:
all_other_news = get_more(plist[1:])# 测试 more
print len(all_other_news)

360


In [59]:
import datetime
a = datetime.datetime.strptime("2014-12-31" + " 23:59:59", "%Y-%m-%d %H:%M:%S")
a

datetime.datetime(2014, 12, 31, 23, 59, 59)

In [60]:
import datetime
from urllib import quote
def date_filter(begin_date, end_date, query_word):
    date0 = datetime.datetime.strptime(begin_date, "%Y-%m-%d")
    date1 = datetime.datetime.strptime(end_date + " 23:59:59", "%Y-%m-%d %H:%M:%S")
    y0, m0, d0 = str(date0.year), str(date0.month), str(date0.day)
    y1, m1, d1 = str(date1.year), str(date1.month), str(date1.day)
    query_word = quote(query_word)
    bt = str(int(time.mktime(date0.timetuple())))
    et = str(int(time.mktime(date1.timetuple())))
    url = 'http://news.baidu.com/ns?from=news&cl=2&bt='+ bt + '&y0='+ y0 +'&m0=' + m0 + '&d0=' + d0 + '&y1=' + y1 + '&m1=' + m1 + '&d1=' + d1 + '&et=' + et + '&q1=' + query_word + '&submit=%B0%D9%B6%C8%D2%BB%CF%C2&q3=&q4=&mt=0&lm=&s=2&begin_date=' + begin_date + '&end_date=' + end_date + '&tn=newsdy&ct1=1&ct=1&rn=20&q6='
    return url

In [61]:
print date_filter('2017-1-2','2017-1-3', '军民')

http://news.baidu.com/ns?from=news&cl=2&bt=1483286400&y0=2017&m0=1&d0=2&y1=2017&m1=1&d1=3&et=1483459199&q1=%E5%86%9B%E6%B0%91&submit=%B0%D9%B6%C8%D2%BB%CF%C2&q3=&q4=&mt=0&lm=&s=2&begin_date=2017-1-2&end_date=2017-1-3&tn=newsdy&ct1=1&ct=1&rn=20&q6=


In [37]:
def page_filter(url, page=0):# append url with pn and rn params
    # rn are set to be 20
    if page is None:
        page = 0
    rn = 20
    pn = page * rn
    rn = str(rn)
    pn = str(pn)
    url = url + '&pn=' + pn
    url = url + '&rn=' + rn
    return url
    

In [40]:
def search_with_page(query_word, page=0):
    url = search(query_word)
    url = page_filter(url, page)
    return url

def date_filter_with_page(begin_date, end_date, query_word, page=0):
    url = date_filter(begin_date, end_date, query_word)
    url = page_filter(url, page)
    return url

In [41]:
print search_with_page('军民融合')
print 
print date_filter_with_page('2017-1-1', '2017-1-2', '军民融合', 2)

http://news.baidu.com/ns?cl=2&tn=news&word=%E5%86%9B%E6%B0%91%E8%9E%8D%E5%90%88&pn=0&rn=20

http://news.baidu.com/ns?from=news&cl=2&bt=1483200000&y0=2017&m0=1&d0=1&y1=2017&m1=1&d1=2&et=1483372799&q1=%E5%86%9B%E6%B0%91%E8%9E%8D%E5%90%88&submit=%B0%D9%B6%C8%D2%BB%CF%C2&q3=&q4=&mt=0&lm=&s=2&begin_date=2017-1-1&end_date=2017-1-2&tn=newsdy&ct1=1&ct=1&pn=40&rn=20


In [29]:
test = search_with_page('军民融合', 3)
soup = crawl(test)
ans = parse(soup)
# print ans

In [31]:
len(ans)

20